In [1]:
import os
import sys
import json
from tqdm import tqdm_notebook as tqdm
from pprint import pprint

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns

## data load

In [2]:
BASE_DIR = '../mnt/inputs/origin'
TRN_BASE_DIR = '../mnt/inputs/features/train'
_TRN_BASE_DIR = '../mnt/inputs/features/_train'
TST_BASE_DIR = '../mnt/inputs/features/test'
# BASE_DIR = '../mnt/inputs/features/test'

In [3]:
!ls $BASE_DIR
!ls $TRN_BASE_DIR
!ls $_TRN_BASE_DIR

data-science-bowl-2019.zip  test.csv	  train2.csv
sample_submission.csv	    train.csv	  train_labels.csv
specs.csv		    train.pkl.gz
specs_w_event_id.csv	    train1.csv
EncodingTitles.pkl  EventCount2.pkl    PrevAssessAccByTitle.pkl  Worldcount.pkl
EventCount.pkl	    KernelBasics2.pkl  SessionTime2.pkl
EncodingTitles.pkl  EventCount2.pkl    PrevAssessAccByTitle.pkl  Worldcount.pkl
EventCount.pkl	    KernelBasics2.pkl  SessionTime2.pkl


In [5]:
df = pd.read_pickle(f'{TRN_BASE_DIR}/KernelBasics2.pkl')
df.head()

,12 Monkeys,Air Show,All Star Sorting,Balancing Act,Bird Measurer (Assessment),Bottle Filler (Activity),Bubble Bath,Bug Measurer (Activity),Cart Balancer (Assessment),Cauldron Filler (Assessment),...,prev_acc_gr_0,prev_acc_gr_1,prev_acc_gr_2,prev_acc_gr_3,prev_cumnum_c,prev_cumnum_inc,prev_num_corrects,prev_num_incorrects,title,type
0,0,0,164,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Mushroom Sorter (Assessment),Assessment
1,0,193,164,0,0,0,0,104,0,0,...,0,0,0,1,1,0,1,0,Bird Measurer (Assessment),Assessment
2,0,193,164,0,92,0,0,104,0,0,...,1,0,0,0,1,11,0,11,Mushroom Sorter (Assessment),Assessment
3,0,193,231,0,92,165,133,104,0,0,...,0,0,0,1,2,11,1,0,Mushroom Sorter (Assessment),Assessment
4,0,337,231,0,92,165,133,185,0,0,...,0,0,1,0,3,12,1,1,Bird Measurer (Assessment),Assessment


In [7]:
df.sort_values(['game_session', 'installation_id']).index

Int64Index([2, 3, 9, 2, 6, 0, 3, 1, 8, 1,
            ...
            1, 1, 5, 0, 1, 7, 0, 0, 5, 1],
           dtype='int64', length=17690)

In [21]:
trn_df = pd.read_pickle(f'{BASE_DIR}/train.pkl.gz')

In [ ]:
(trn_df == trn_df.sort_values('timestamp')).all()

In [10]:
for filename in ['EventCount.pkl', 'EventCount2.pkl',  'KernelBasics2.pkl', 'EncodingTitles.pkl',  'EventCount2.pkl', 'PrevAssessAccByTitle.pkl', 'Worldcount.pkl']:
    df1 = pd.read_pickle(f'{TRN_BASE_DIR}/{filename}')
    df2 = pd.read_pickle(f'{_TRN_BASE_DIR}/{filename}')
    
    assert (df1.columns == df2.columns).all()
    for col in df1.columns:
        print(f'------------------------------------------- {col} ----------------------------------------------')
        print((df1[col] == df2[col]).all())

------------------------------------------- ev_cnt2000 ----------------------------------------------
True
------------------------------------------- ev_cnt2010 ----------------------------------------------
True
------------------------------------------- ev_cnt2020 ----------------------------------------------
True
------------------------------------------- ev_cnt2025 ----------------------------------------------
True
------------------------------------------- ev_cnt2030 ----------------------------------------------
True
------------------------------------------- ev_cnt2035 ----------------------------------------------
True
------------------------------------------- ev_cnt2040 ----------------------------------------------
True
------------------------------------------- ev_cnt2050 ----------------------------------------------
True
------------------------------------------- ev_cnt2060 ----------------------------------------------
True
-------------------------------------

True
------------------------------------------- ev_cnt6f4bd64e ----------------------------------------------
True
------------------------------------------- ev_cnt6f8106d9 ----------------------------------------------
True
------------------------------------------- ev_cnt7040c096 ----------------------------------------------
True
------------------------------------------- ev_cnt709b1251 ----------------------------------------------
True
------------------------------------------- ev_cnt71e712d8 ----------------------------------------------
True
------------------------------------------- ev_cnt71fe8f75 ----------------------------------------------
True
------------------------------------------- ev_cnt731c0cbe ----------------------------------------------
True
------------------------------------------- ev_cnt736f9581 ----------------------------------------------
True
------------------------------------------- ev_cnt7372e1a5 ----------------------------------------------
T

True
------------------------------------------- ev_cntdf4fe8b6 ----------------------------------------------
True
------------------------------------------- ev_cnte04fb33d ----------------------------------------------
True
------------------------------------------- ev_cnte080a381 ----------------------------------------------
True
------------------------------------------- ev_cnte37a2b78 ----------------------------------------------
True
------------------------------------------- ev_cnte3ff61fb ----------------------------------------------
True
------------------------------------------- ev_cnte4d32835 ----------------------------------------------
True
------------------------------------------- ev_cnte4f1efe6 ----------------------------------------------
True
------------------------------------------- ev_cnte5734469 ----------------------------------------------
True
------------------------------------------- ev_cnte57dd7af ----------------------------------------------
T

True
------------------------------------------- game_session ----------------------------------------------
True
------------------------------------------- title_enc ----------------------------------------------
False
------------------------------------------- ev_cnt0086365d ----------------------------------------------
True
------------------------------------------- ev_cnt00c73085 ----------------------------------------------
True
------------------------------------------- ev_cnt01ca3a3c ----------------------------------------------
True
------------------------------------------- ev_cnt022b4259 ----------------------------------------------
True
------------------------------------------- ev_cnt02a42007 ----------------------------------------------
True
------------------------------------------- ev_cnt0330ab6a ----------------------------------------------
True
------------------------------------------- ev_cnt0413e89d ----------------------------------------------
True
--

True
------------------------------------------- ev_cnt9d4e7b25 ----------------------------------------------
True
------------------------------------------- ev_cnt9de5e594 ----------------------------------------------
True
------------------------------------------- ev_cnt9e34ea74 ----------------------------------------------
True
------------------------------------------- ev_cnt9e4c8c7b ----------------------------------------------
True
------------------------------------------- ev_cnt9e6b7fb5 ----------------------------------------------
True
------------------------------------------- ev_cnt9ed8f6da ----------------------------------------------
True
------------------------------------------- ev_cnt9ee1c98c ----------------------------------------------
True
------------------------------------------- ev_cnta0faea5d ----------------------------------------------
True
------------------------------------------- ev_cnta1192f43 ----------------------------------------------
T

True
------------------------------------------- cum_num_correct_Chest Sorter (Assessment) ----------------------------------------------
True
------------------------------------------- cum_num_correct_Mushroom Sorter (Assessment) ----------------------------------------------
True
------------------------------------------- cum_num_incorrect_Bird Measurer (Assessment) ----------------------------------------------
True
------------------------------------------- cum_num_incorrect_Cart Balancer (Assessment) ----------------------------------------------
True
------------------------------------------- cum_num_incorrect_Cauldron Filler (Assessment) ----------------------------------------------
True
------------------------------------------- cum_num_incorrect_Chest Sorter (Assessment) ----------------------------------------------
True
------------------------------------------- cum_num_incorrect_Mushroom Sorter (Assessment) ----------------------------------------------
True
--------

In [16]:
df1.head()

,game_session,installation_id,title_type_cnt_12 Monkeys_Clip,title_type_cnt_Air Show_Game,title_type_cnt_All Star Sorting_Game,title_type_cnt_Balancing Act_Clip,title_type_cnt_Bird Measurer (Assessment)_Assessment,title_type_cnt_Bottle Filler (Activity)_Activity,title_type_cnt_Bubble Bath_Game,title_type_cnt_Bug Measurer (Activity)_Activity,...,wrd_type_cnt_CRYSTALCAVES_Game,wrd_type_cnt_MAGMAPEAK_Activity,wrd_type_cnt_MAGMAPEAK_Assessment,wrd_type_cnt_MAGMAPEAK_Clip,wrd_type_cnt_MAGMAPEAK_Game,wrd_type_cnt_NONE_Clip,wrd_type_cnt_TREETOPCITY_Activity,wrd_type_cnt_TREETOPCITY_Assessment,wrd_type_cnt_TREETOPCITY_Clip,wrd_type_cnt_TREETOPCITY_Game
0,901acc108f55a5a1,0006a69f,1,0,153,0,0,0,0,0,...,0,102,0,2,131,2,250,0,7,153
1,77b8ee947eb84b4e,0006a69f,1,180,153,0,0,0,0,204,...,0,102,0,2,131,2,454,48,10,394
2,6bdf9623adc94d89,0006a69f,1,180,153,0,87,0,0,204,...,0,102,0,2,131,2,454,135,10,394
3,e7e7db2a241eadcc,0006a69f,2,180,203,0,87,110,115,204,...,0,322,0,5,401,3,781,170,16,444
4,9501794defd84e4d,0006a69f,2,180,203,0,87,110,115,204,...,0,322,0,5,401,3,781,187,16,444


In [17]:
df2.head()

,game_session,installation_id,title_type_cnt_12 Monkeys_Clip,title_type_cnt_Air Show_Game,title_type_cnt_All Star Sorting_Game,title_type_cnt_Balancing Act_Clip,title_type_cnt_Bird Measurer (Assessment)_Assessment,title_type_cnt_Bottle Filler (Activity)_Activity,title_type_cnt_Bubble Bath_Game,title_type_cnt_Bug Measurer (Activity)_Activity,...,wrd_type_cnt_CRYSTALCAVES_Game,wrd_type_cnt_MAGMAPEAK_Activity,wrd_type_cnt_MAGMAPEAK_Assessment,wrd_type_cnt_MAGMAPEAK_Clip,wrd_type_cnt_MAGMAPEAK_Game,wrd_type_cnt_NONE_Clip,wrd_type_cnt_TREETOPCITY_Activity,wrd_type_cnt_TREETOPCITY_Assessment,wrd_type_cnt_TREETOPCITY_Clip,wrd_type_cnt_TREETOPCITY_Game
0,901acc108f55a5a1,0006a69f,1,0,153,0,0,0,0,0,...,0,102,0,2,131,2,250,0,7,153
1,77b8ee947eb84b4e,0006a69f,1,180,153,0,0,0,0,204,...,0,102,0,2,131,2,454,48,10,394
2,6bdf9623adc94d89,0006a69f,1,180,153,0,87,0,0,204,...,0,102,0,2,131,2,454,135,10,394
3,e7e7db2a241eadcc,0006a69f,2,180,203,0,87,110,115,204,...,0,322,0,5,401,3,781,170,16,444
4,9501794defd84e4d,0006a69f,2,180,203,0,87,110,115,204,...,0,322,0,5,401,3,781,187,16,444


In [18]:
f'{TRN_BASE_DIR}/{filename}', f'{_TRN_BASE_DIR}/{filename}'

('../mnt/inputs/features/train/Worldcount.pkl',
 '../mnt/inputs/features/_train/Worldcount.pkl')

In [20]:
(df1 == df2).mean()

game_session                            1.0
installation_id                         1.0
title_type_cnt_12 Monkeys_Clip          1.0
title_type_cnt_Air Show_Game            1.0
title_type_cnt_All Star Sorting_Game    1.0
                                       ... 
wrd_type_cnt_NONE_Clip                  1.0
wrd_type_cnt_TREETOPCITY_Activity       1.0
wrd_type_cnt_TREETOPCITY_Assessment     1.0
wrd_type_cnt_TREETOPCITY_Clip           1.0
wrd_type_cnt_TREETOPCITY_Game           1.0
Length: 63, dtype: float64

In [11]:
df1 = pd.read_csv('../preprocessed_train1.csv')
df2 = pd.read_csv('../preprocessed_train2.csv')

In [20]:
(df1 == df2).all().mean()

for col in df1.columns:
    print(f'------------------------------------------- {col} ----------------------------------------------')
    print((df1[col].fillna(0) == df2[col].fillna(0)).all())

------------------------------------------- 12 Monkeys ----------------------------------------------
True
------------------------------------------- Air Show ----------------------------------------------
True
------------------------------------------- All Star Sorting ----------------------------------------------
True
------------------------------------------- Balancing Act ----------------------------------------------
True
------------------------------------------- Bird Measurer (Assessment) ----------------------------------------------
True
------------------------------------------- Bottle Filler (Activity) ----------------------------------------------
True
------------------------------------------- Bubble Bath ----------------------------------------------
True
------------------------------------------- Bug Measurer (Activity) ----------------------------------------------
True
------------------------------------------- Cart Balancer (Assessment) ----------------------

True
------------------------------------------- ev_cnt160654fd ----------------------------------------------
True
------------------------------------------- ev_cnt16667cc5 ----------------------------------------------
True
------------------------------------------- ev_cnt16dffff1 ----------------------------------------------
True
------------------------------------------- ev_cnt17113b36 ----------------------------------------------
True
------------------------------------------- ev_cnt17ca3959 ----------------------------------------------
True
------------------------------------------- ev_cnt19967db1 ----------------------------------------------
True
------------------------------------------- ev_cnt1996c610 ----------------------------------------------
True
------------------------------------------- ev_cnt1af8be29 ----------------------------------------------
True
------------------------------------------- ev_cnt1b54d27f ----------------------------------------------
T

True
------------------------------------------- ev_cnt7ec0c298 ----------------------------------------------
True
------------------------------------------- ev_cnt7f0836bf ----------------------------------------------
True
------------------------------------------- ev_cnt7fd1ac25 ----------------------------------------------
True
------------------------------------------- ev_cnt804ee27f ----------------------------------------------
True
------------------------------------------- ev_cnt828e68f9 ----------------------------------------------
True
------------------------------------------- ev_cnt832735e1 ----------------------------------------------
True
------------------------------------------- ev_cnt83c6c409 ----------------------------------------------
True
------------------------------------------- ev_cnt84538528 ----------------------------------------------
True
------------------------------------------- ev_cnt84b0e0c8 ----------------------------------------------
T

True
------------------------------------------- ev_cntf56e0afc ----------------------------------------------
True
------------------------------------------- ev_cntf5b8c21a ----------------------------------------------
True
------------------------------------------- ev_cntf6947f54 ----------------------------------------------
True
------------------------------------------- ev_cntf71c4741 ----------------------------------------------
True
------------------------------------------- ev_cntf7e47413 ----------------------------------------------
True
------------------------------------------- ev_cntf806dc10 ----------------------------------------------
True
------------------------------------------- ev_cntf93fc684 ----------------------------------------------
True
------------------------------------------- ev_cntfbaf3456 ----------------------------------------------
True
------------------------------------------- ev_cntfcfdffb6 ----------------------------------------------
T

In [22]:
df1['title_enc'][df1['title_enc'] != df2['title_enc']]

0         9
1        37
2         4
3         9
4         4
         ..
17685     6
17686    21
17687     6
17688     4
17689     9
Name: title_enc, Length: 17690, dtype: int64

In [23]:
df2['title_enc'][df1['title_enc'] != df2['title_enc']]

0        41
1        10
2        15
3        41
4        15
         ..
17685    21
17686     1
17687    21
17688    15
17689    41
Name: title_enc, Length: 17690, dtype: int64

In [25]:
(df1['title_enc'] != df2['title_enc']).mean()

1.0

In [26]:
df1.to_csv('../mnt/inputs/origin/train1.csv', index=False)
df2.to_csv('../mnt/inputs/origin/train2.csv', index=False)

In [27]:
df1.title_enc.value_counts()

21    4151
6     4055
9     3757
37    2981
4     2746
Name: title_enc, dtype: int64

In [29]:
df = pd.read_pickle(f'{TRN_BASE_DIR}/EncodingTitles.pkl')
df

,installation_id,game_session,title_enc
0,0006a69f,901acc108f55a5a1,9
1,0006a69f,77b8ee947eb84b4e,4
2,0006a69f,6bdf9623adc94d89,9
3,0006a69f,e7e7db2a241eadcc,9
4,0006a69f,9501794defd84e4d,9
...,...,...,...
0,ffe3b25d,29e0d2a854ca11d0,6
1,ffe3b25d,35af4f36098afcf4,6
0,ffeb0b1b,dadd1a4d8ac68ab0,6
1,ffeb0b1b,a6885ab824fbc32c,9


In [30]:
df = pd.read_pickle('../mnt/inputs/features/KernelBasics2.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '../mnt/inputs/features/KernelBasics2.pkl'

In [31]:
df = pd.read_pickle('../mnt/inputs/features/train/KernelBasics2.pkl')

In [32]:
df.head()

,12 Monkeys,Air Show,All Star Sorting,Balancing Act,Bird Measurer (Assessment),Bottle Filler (Activity),Bubble Bath,Bug Measurer (Activity),Cart Balancer (Assessment),Cauldron Filler (Assessment),...,prev_acc_gr_0,prev_acc_gr_1,prev_acc_gr_2,prev_acc_gr_3,prev_cumnum_c,prev_cumnum_inc,prev_num_corrects,prev_num_incorrects,title,type
0,0,0,164,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Mushroom Sorter (Assessment),Assessment
1,0,193,164,0,0,0,0,104,0,0,...,0,0,0,1,1,0,1,0,Bird Measurer (Assessment),Assessment
2,0,193,164,0,92,0,0,104,0,0,...,1,0,0,0,1,11,0,11,Mushroom Sorter (Assessment),Assessment
3,0,193,231,0,92,165,133,104,0,0,...,0,0,0,1,2,11,1,0,Mushroom Sorter (Assessment),Assessment
4,0,337,231,0,92,165,133,185,0,0,...,0,0,1,0,3,12,1,1,Bird Measurer (Assessment),Assessment


In [33]:
'Bird Measurer (Assessment)' in df.columns

True